### ETAのシミュレーション
自動運転車が道路を走る際、

Strategic（運行前計画） => Tactical（運行時の計画） => Ground（細かいセンシングなど）

の3つのフェーズがある。本研究では2つ目のTacticalフェーズにおける、車両の間隔制御を目的とする。まず、Strategicフェーズでは、だいたい何時頃に何台が到着するかがわかる。それを受けてTacticalフェーズでは実際に道路に到着した車に走路の事前予約（以下、CWP Reservation）を行わせ、その通りに運行する。このシミュレーションはCWP Reservationのアルゴリズムについての研究である。

ここでは大きく3つのオブジェクトが登場する

1. CWPTable: 予約管理をするテーブル
2. Car: 到着時刻に基づきCWPTableに希望時刻を打刻する。
3. Simulation: 帯域密度や到着間隔の管理を行う。

In [10]:
import random
import sys
sys.path.append("../../../")
from simulation_classes.eta.car import Cars
from simulation_classes.eta.CWPTable import CWPTable

In [11]:
# 初期設定（独立変数群）
CAR_NUM = 4 #時間窓内の車の総数
START_TIME = 0
GLOBAL_TIME_WINDOW = 20 # Strategic Phaseでの時間幅
TOTAL_LENGTH = 2000 # 入り口から出口までの距離 [m]
WAYPOINTS_NUM = 20 #constrained way pointの数
MEAN_SPEED = 30 # 秒速[m/s]として定義
SCALE_FACTOR = 20 #シミュレーション上で距離を変換するための係数
TIME_STEP = 1
TOTAL_TIME = 1000

# 従属的に決まるもの
SIMULATION_STEPS = int(TOTAL_TIME / TIME_STEP)
WAYPOINTS = [{"waypoint_idx": i+1, "x": TOTAL_LENGTH / WAYPOINTS_NUM * (i+1)} for i in range(WAYPOINTS_NUM)]


arrival_times = sorted([random.uniform(0, GLOBAL_TIME_WINDOW) for _ in range(CAR_NUM)])

CARS = [Cars(arrival_time=time, index=index, mean_speed=MEAN_SPEED) for index, time in enumerate(arrival_times)]
# for i in range(CAR_NUM):
    

In [12]:
# 一旦雑にsimulation開始
cwp_table = CWPTable(waypoints = WAYPOINTS)

for i in range(SIMULATION_STEPS):
    t = i * TIME_STEP

    for car in CARS:
        # この時間に到着する車がいれば打刻する
        if car.arrival_time >= t and car.arrival_time < t + TIME_STEP:
            desired_list = car.create_desired_list(WAYPOINTS)
            # print(desired_list)
            is_valid = cwp_table.validate(desired_list)
            if is_valid:
                cwp_table.register(desired_list)
            else:
                calibrated_list = cwp_table.calibrate_list(desired_list)
                cwp_table.register(calibrated_list)
            continue
        
        # そうでない車は普通に進む
        car.proceed(time_step=TIME_STEP)
            
                




初回


In [22]:
df = cwp_table.waypoint_table
print(df[df['car_idx'] == 0])
print(df[df['car_idx'] == 1])

    waypoint_idx       x        eta  car_idx
0              1   100.0   7.339641        0
1              2   200.0  10.672974        0
2              3   300.0  14.006307        0
3              4   400.0  17.339641        0
4              5   500.0  20.672974        0
5              6   600.0  24.006307        0
6              7   700.0  27.339641        0
7              8   800.0  30.672974        0
8              9   900.0  34.006307        0
9             10  1000.0  37.339641        0
10            11  1100.0  40.672974        0
11            12  1200.0  44.006307        0
12            13  1300.0  47.339641        0
13            14  1400.0  50.672974        0
14            15  1500.0  54.006307        0
15            16  1600.0  57.339641        0
16            17  1700.0  60.672974        0
17            18  1800.0  64.006307        0
18            19  1900.0  67.339641        0
19            20  2000.0  70.672974        0
    waypoint_idx       x        eta  car_idx
0         